In [ ]:
from models.resnet import ResNet34
import torch
from collections import OrderedDict
from torchvision.transforms import transforms
import torchvision
from torch.utils.data import DataLoader
import numpy as np
import sklearn.covariance
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from joblib import load ,dump
device = "cuda:0"

model = ResNet34(10).to(device)
checkpoint = torch.load("./pre_trained/resnet34_cifar10.pth")
new_state_dict = OrderedDict()
for k,v in checkpoint.items():
    new_key = k.replace('module.','')
    new_state_dict[new_key] = v
model.load_state_dict(new_state_dict)


"""
ID data -> Cifar 10
Profiling(mean and covariance is done on Cifar 10)
"""
data_path = "./data"

transform_test = transforms.Compose([
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))  # Normalize using CIFAR-10 mean and std
])
test_dataset = torchvision.datasets.CIFAR10(
    root='./data',  # Path to the dataset folder
    train=False,    # Indicate this is the test dataset
    download=True,  # Download the dataset if not available
    transform=transform_test  # Apply the transformations
)

train_dataset = torchvision.datasets.CIFAR10(
    root='./data',  # Path to the dataset folder
    train=True,    # Indicate this is the test dataset
    download=True,  # Download the dataset if not available
    transform=transform_test  # Apply the transformations
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=64,   # Set the batch size (you can adjust this)
    shuffle=False,   # Do not shuffle for testing
    num_workers=2    # Number of subprocesses to use for data loading
)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=64,   # Set the batch size (you can adjust this)
    shuffle=False,   # Do not shuffle for testing
    num_workers=2    # Number of subprocesses to use for data loading
)

# Create test datasets
svhn_test_dataset = torchvision.datasets.SVHN(root=data_path, split='test', download=True, transform=transform_test)
isun_test_dataset = torchvision.datasets.ImageFolder(root=f"{data_path}/isun", transform=transform_test)
lsun_test_dataset = torchvision.datasets.ImageFolder(root=f"{data_path}/lsun_resize", transform=transform_test)

# Create test data loaders
batch_size = 64  # Set your batch size here
svhn_test_loader = DataLoader(svhn_test_dataset, batch_size=batch_size, shuffle=False)
isun_test_loader = DataLoader(isun_test_dataset, batch_size=batch_size, shuffle=False)
lsun_test_loader = DataLoader(lsun_test_dataset, batch_size=batch_size, shuffle=False)

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------
autoAttack_data = torch.load('./adv_samples/densenet3_cifar10_autoattack.pt')
data_list, label_list = zip(*autoAttack_data)
inputs_tensor = torch.stack([torch.from_numpy(data).float() for data in data_list])
labels_tensor = torch.tensor(label_list, dtype=torch.long)

AA_dataset = TensorDataset(inputs_tensor, labels_tensor)
Autoattack_test_loader = DataLoader(AA_dataset, batch_size=64, shuffle=False)
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------
fgsm_data = torch.load('./adv_samples/densenet3_cifar10_fgsm.pt')
data_list, label_list = zip(*fgsm_data)
inputs_tensor = torch.stack([torch.from_numpy(data).float() for data in data_list])
labels_tensor = torch.tensor(label_list, dtype=torch.long)
fsgm_dataset = TensorDataset(inputs_tensor, labels_tensor)
fgsm_test_loader = DataLoader(fsgm_dataset, batch_size=64, shuffle=False)
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------
pgd_data = torch.load('./adv_samples/densenet3_cifar10_pgd.pt')
data_list, label_list = zip(*pgd_data)
inputs_tensor = torch.stack([torch.from_numpy(data).float() for data in data_list])
labels_tensor = torch.tensor(label_list, dtype=torch.long)
pgd_dataset = TensorDataset(inputs_tensor, labels_tensor)
pgd_test_loader = DataLoader(pgd_dataset, batch_size=64, shuffle=False)
# """
# extract Features -> get features of a specified dataset , using a model, and at a specific index

# """
# def extract_features(loader, model):
#     layer_index=4
#     features0 = []
#     features1 = []
#     features2 = []
#     features3 = []
#     features4 = []
#     labels = []
#     model.eval()
#     count = 0
#     with torch.no_grad():
#         for inputs, targets in loader:
#             count += 1
#             inputs = inputs.cuda()
#             layer_features4,layer_features3,layer_features2,layer_features1,layer_features0 = model.intermediate_forward(inputs, layer_index=layer_index)
            
#             layer_features0_flat = layer_features1.view(layer_features0.size(0), -1)
#             features0.append(layer_features0_flat.cpu())

#             layer_features1_flat = layer_features1.view(layer_features1.size(0), -1)
#             features1.append(layer_features1_flat.cpu())

#             layer_features2_flat = layer_features2.view(layer_features2.size(0), -1)
#             features2.append(layer_features2_flat.cpu())
            
#             layer_features3_flat = layer_features3.view(layer_features3.size(0), -1)
#             features3.append(layer_features3_flat.cpu())

#             layer_features4_flat = layer_features1.view(layer_features4.size(0), -1)
#             features4.append(layer_features4_flat.cpu())

#             labels.append(targets.cpu())
#                 # break
#         features0 = torch.cat(features0, dim=0).numpy()
#         features1 = torch.cat(features1, dim=0).numpy()
#         features2 = torch.cat(features2, dim=0).numpy()
#         features3 = torch.cat(features3, dim=0).numpy()
#         features4 = torch.cat(features4, dim=0).numpy()
#         labels = torch.cat(labels, dim=0).numpy() if len(labels) > 0 else None
#     return features0 ,features1,features2, features3,features4,labels


# ood_features_dict = {}

test_loaders = {
    "ID": test_loader,
    "ID_Train": train_loader,
    "PGD": pgd_test_loader,
    "FGSM": fgsm_test_loader,
    "LSUN": lsun_test_loader,
    "SVHN": svhn_test_loader,
    "iSUN":isun_test_loader,
    "AutoAttack" : Autoattack_test_loader,
}
# # Initialize the dictionary to store features and labels

# # Loop over each dataset
# for dataset_name, loader in test_loaders.items():
#     print(f"Processing dataset: {dataset_name}")
#     features_dict = {}
#     features_dict[dataset_name] = {}
    
   
#     features0,features1,features2,features3,features4, labels = extract_features(loader, model)
    
#     # Store the extracted features and labels in the dictionary
#     features_dict[dataset_name][0] = {
#         "features": features0,
#         "labels": labels
#     }
#     features_dict[dataset_name][1] = {
#         "features": features1,
#         "labels": labels
#     }
#     features_dict[dataset_name][2] = {
#         "features": features2,
#         "labels": labels
#     }
#     features_dict[dataset_name][3] = {
#         "features": features3,
#         "labels": labels
#     }
#     features_dict[dataset_name][4] = {
#         "features": features4,
#         "labels": labels
#     }
    
#     # print("Feature extraction completed.")

#     dump(features_dict,f"extracted_features_{dataset_name}.pkl")





In [ ]:
# loaded_features_dict = {}

#     loaded_features_dict[dataset_name] = load(f"extracted_features_{dataset_name}.pkl")


In [ ]:
from joblib import load ,dump
# print("Feature extraction completed.")

# dump(features_dict0,"extracted_features_all.pkl")
# dump(features_dict0,"extracted_features_all.pkl")
# dump(features_dict0,"extracted_features_all.pkl")

# import cudf
# loaded_features_dict = load("extracted_features_100.pkl")

In [1]:
from joblib import load ,dump
loaded_features_dict = load("extracted_features_ID_Train.pkl")


In [2]:
a =loaded_features_dict['ID_Train'].keys()
print(a)

dict_keys([0, 1, 2, 3, 4])


In [3]:
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.decomposition import PCA,IncrementalPCA
import umap 

gmm_final = {} 
umap_final = {}
new_dim = {}
for layer_index in loaded_features_dict['ID_Train'].keys():
    # umap_model = umap.UMAP(n_components=700, n_neighbors=700,min_dist=0.5)
    gmm_final[layer_index]= {}
    gmm = GaussianMixture(n_components=10)
    print(layer_index, "----------->")
    print("fitting the umap model ")
    feature_values = loaded_features_dict['ID_Train'][layer_index]["features"]
    print("from len = ",len(feature_values[0]))
    pca = PCA(n_components=200)
    pca = pca.fit(feature_values)
    feature_values = pca.transform(feature_values)
    print(" values reduced to ", len(feature_values[0]))
    gmm.fit(feature_values)
    gmm_final[layer_index]= gmm

        

/home/local/ASUAD/asing651/.conda/envs/rapids-24.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0 ----------->
fitting the umap model 
from len =  65536
 values reduced to  200
1 ----------->
fitting the umap model 
from len =  65536
 values reduced to  200


KeyboardInterrupt: 

GMM scores

In [ ]:
a= np.array([1,2,3,4,5,6,7,8,9])
a =a.reshape(3,3)
print(a)

In [ ]:
print(gmm_final)
test_loaders = ["ID" ,    "PGD" ,    "FGSM" ,    "LSUN",    "SVHN" ,    "iSUN" ,    "AutoAttack" ]

In [ ]:

from sklearn.preprocessing import StandardScaler
gmm_scores = {}
for data in test_loaders:
   gmm_scores[data] = {}
   loaded_features_dict = load(f"extracted_features_{data}.pkl")
   for layer_index in loaded_features_dict[data].keys():
      gmm =  gmm_final[layer_index]
   #    print(loaded_features_dict[data][layer_index])
      
      feature_values = loaded_features_dict[data][layer_index]['features']
      print(data," -... ")
      pca = PCA(n_components=200)
      feature_values =  pca.fit_transform(feature_values)
      print("converted to lower dimenion")
      print(feature_values[0].shape)
      gmm_scores[data][layer_index] = gmm.score_samples(feature_values)
      print(f"score generated for {layer_index}, for data = {data}")


In [ ]:

len(gmm_scores['SVHN'][0])

In [ ]:
import pandas as pd
rows = []
for data_type, layers in gmm_scores.items():
    for i in range(len(next(iter(layers.values())))):  # Assumes all layers have the same length
        row = [data_type]
        for layer_index in range(5):  # Assuming 5 layer indices
            row.append(layers[layer_index][i])
        rows.append(row)

# Create DataFrame
headers = ["data_type", "layer_index1", "layer_index2", "layer_index3", "layer_index4", "layer_index5"]
df = pd.DataFrame(rows, columns=headers)
df.to_csv("gmm_scores11thdec.csv", index=False)
# Display the DataFrame
print(df)

In [ ]:

import pandas as pd 
id_df = df[df['data_type'] == 'ID'] 
thresholds = {}
layer_columns = ["layer_index1", "layer_index2", "layer_index3", "layer_index4", "layer_index5"]
for col in layer_columns:
    threshold = id_df[col].quantile(0.05)  # 5th percentile
    thresholds[col] = threshold

print("Thresholds for 95% TPR:")
for layer, thresh in thresholds.items():
    print(f"{layer}: {thresh}")


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

def compute_metrics_for_data_type(df, thresholds, test_data_type):
    """
    Computes Accuracy, AUROC, AUPR for each layer and adds a column counting
    the number of layers that correctly classified each sample.

    Parameters:
    - df (pd.DataFrame): The dataset containing 'data_type' and layer scores.
    - thresholds (dict): Dictionary with layer names as keys and threshold values as values.
    - test_data_type (str): The specific OOD data type to evaluate against 'ID'.

    Returns:
    - metrics_result (dict): Dictionary containing metrics for each layer.
    - subset (pd.DataFrame): The subset DataFrame with the new 'correct_layer_count' column.
    """
    # Filter for ID and the test_data_type
    subset = df[(df['data_type'] == 'ID') | (df['data_type'] == test_data_type)].copy()
    
    # Create binary ground truth: 1 for ID, 0 for non-ID
    y_true = (subset['data_type'] == 'ID').astype(int)
    
    metrics_result = {}
    
    # Initialize a DataFrame to store binary predictions for each layer
    predictions = pd.DataFrame(index=subset.index)
    
    for layer_col, threshold in thresholds.items():
        # Extract scores for this layer
        y_scores = subset[layer_col].values
        
        # Binary predictions based on threshold
        y_pred = (y_scores >= threshold).astype(int)
        
        # Store predictions in the DataFrame
        predictions[layer_col] = y_pred
        
        
        acc = accuracy_score(y_true, y_pred)

        # Compute AUROC
        try:
            auroc = roc_auc_score(y_true, y_scores)
        except ValueError:
            auroc = float('nan')
        
        # Compute AUPR
        try:
            aupr = average_precision_score(y_true, y_scores)
        except ValueError:
            aupr = float('nan')
        
        # Store metrics for this layer
        metrics_result[layer_col] = {
            'accuracy': acc,
            'auroc': auroc,
            'aupr': aupr
        }
    correct_predictions = predictions.eq(y_true, axis=0)  # DataFrame of booleans
    subset['correct_layer_count'] = correct_predictions.sum(axis=1)
    
    return metrics_result, subset

# Example usage:
# Assume we have a DataFrame 'df' and a dictionary 'thresholds' defined elsewhere
# Replace 'OOD' with whatever non-ID data_type you want to evaluate
test_data_types = ['SVHN','iSUN','LSUN','AutoAttack','PGD','FGSM','ID']

# Initialize a dictionary to store all metrics for each test_data_type
all_metrics = {}

for d_type in test_data_types:
    metrics_for_ood, subset_with_counts = compute_metrics_for_data_type(df, thresholds, d_type)
    all_metrics[d_type] = metrics_for_ood
    print(f"\nData Type: {d_type}")
    for layer, vals in metrics_for_ood.items():
        print(f"{layer} vs ID:")
        print(f"  Accuracy: {vals['accuracy']:.4f}  ,  AUROC: {vals['auroc']:.4f}  ,  AUPR: {vals['aupr']:.4f}")
    


df.loc[subset_with_counts.index, 'correct_layer_count_' + d_type] = subset_with_counts['correct_layer_count']

# After the loop, 'df' will have new columns like 'correct_layer_count_SVHN', etc.


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# Assuming 'df' is your DataFrame
df_long = df.melt(id_vars='data_type', 
                  var_name='layer', 
                  value_name='score')

# List of layer columns
layers = ["layer_index1", "layer_index2", "layer_index3", "layer_index4", "layer_index5"]

for layer in layers:
   
    filtered_df = df
    plt.figure(figsize=(8, 6))
    sns.scatterplot(data=filtered_df, x='data_type', y=layer, hue='data_type', palette='deep', alpha=0.7)
    plt.title(f'Scatter Plot for {layer}')
    plt.xlabel('Data Type')
    plt.ylabel('Score')
    plt.legend(title='Data Type')
    plt.show()




In [ ]:
# from sklearn.manifold import TSNE
# import numpy as np

# # Define the new dictionary for t-SNE representations
# tsne_features_dict = {}

# # Set t-SNE parameters (you can adjust these as needed)
# tsne = TSNE(n_components=2, random_state=42)

# # Iterate through test loaders
# for test_loader_name, layer_data in loaded_features_dict.items():
#     tsne_features_dict[test_loader_name] = {}
    
#     for layer_index, layer_info in layer_data.items():
#         # Get features
#         features = layer_info["features"]
        
#         # Ensure features are in a compatible format (e.g., numpy array)
#         features = np.array(features)
        
#         # Compute t-SNE
#         tsne_representation = tsne.fit_transform(features)
        
#         # Save t-SNE results to the new dictionary
#         tsne_features_dict[test_loader_name][layer_index] = {
#             "tsne_representation": tsne_representation
#         }

# # Now tsne_features_dict contains t-SNE representations for each test_loader and layer_index


In [ ]:
# import torch
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE

# # Assume `model` is your ResNet instance and `train_loader` is your DataLoader with 100 samples from 10 classes.

# # Initialize storage for features and labels
# features_0 = []
# labels = []
# layer_index = 0
# # Extract features from layer 1
# model.eval()
# with torch.no_grad():
#     for inputs, targets in train_loader:
#         inputs = inputs.cuda()
#         layer1_features = model.intermediate_forward(inputs, layer_index=layer_index)
#         # Flatten the features if they are multi-dimensional (e.g., [batch, channels, height, width])
#         layer1_features_flat = layer1_features.view(layer1_features.size(0), -1)
#         features_0.append(layer1_features_flat.cpu())
#         labels.append(targets.cpu())
#         if len(labels) > 500:
#             break

# # Concatenate all features and labels
# features_0 = torch.cat(features_0, dim=0).numpy()
# labels = torch.cat(labels, dim=0).numpy()

# # PCA and t-SNE feature reduction
# pca = PCA(n_components=2)
# pca_features = pca.fit_transform(features_0)

# tsne = TSNE(n_components=2, perplexity=30, random_state=0)
# tsne_features = tsne.fit_transform(features_0)

# # Plotting all features together
# fig, axes = plt.subplots(1, 2, figsize=(15, 7))

# # PCA Plot
# axes[0].scatter(pca_features[:, 0], pca_features[:, 1], c=labels, cmap='tab10', alpha=0.7)
# axes[0].set_title("PCA Feature Visualization")
# axes[0].set_xlabel("Dimension 1")
# axes[0].set_ylabel("Dimension 2")

# # t-SNE Plot
# axes[1].scatter(tsne_features[:, 0], tsne_features[:, 1], c=labels, cmap='tab10', alpha=0.7)
# axes[1].set_title("t-SNE Feature Visualization")
# axes[1].set_xlabel("Dimension 1")
# axes[1].set_ylabel("Dimension 2")

# plt.tight_layout()
# plt.show()




In [ ]:
# # Plotting each class separately
# unique_labels = sorted(set(labels))
# num_classes = len(unique_labels)

# # Create separate plots for each class
# fig, axes = plt.subplots(num_classes, 2, figsize=(15, 5 * num_classes))
# if num_classes == 1:
#     axes = [axes]  # Ensure axes is iterable for a single class

# for i, label in enumerate(unique_labels):
#     class_mask = labels == label
#     class_pca_features = pca_features[class_mask]
#     class_tsne_features = tsne_features[class_mask]
#     # PCA Plot for this class
#     axes[i][0].scatter(class_pca_features[:, 0], class_pca_features[:, 1], cmap='tab10', alpha=0.7)
#     axes[i][0].set_title(f"PCA Feature Visualization - Class {label} , layer = {layer_index}")
#     axes[i][0].set_xlabel("Dimension 1")
#     axes[i][0].set_ylabel("Dimension 2")

#     # t-SNE Plot for this class
#     axes[i][1].scatter(class_tsne_features[:, 0], class_tsne_features[:, 1], cmap='tab10', alpha=0.7)
#     axes[i][1].set_title(f"t-SNE Feature Visualization - Class {label}")
#     axes[i][1].set_xlabel("Dimension 1")
#     axes[i][1].set_ylabel("Dimension 2")

# plt.tight_layout()
# plt.show()

In [ ]:
# import torch
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE

# # Initialize storage for OOD features from all test loaders
# ood_features_dict = {}

# # Helper function to extract features
# def extract_features(loader, model, layer_index=0):
#     features = []
#     labels = []
#     model.eval()
#     with torch.no_grad():
#         for inputs, targets in loader:
#             inputs = inputs.cuda()
#             layer_features = model.intermediate_forward(inputs, layer_index=layer_index)
#             layer_features_flat = layer_features.view(layer_features.size(0), -1)
#             features.append(layer_features_flat.cpu())
#             labels.append(targets.cpu())
#         features = torch.cat(features, dim=0).numpy()
#         labels = torch.cat(labels, dim=0).numpy() if len(labels) > 0 else None
#     return features, labels

# # Extract features for original dataset (in-distribution features)
# features_id, labels_id = extract_features(train_loader, model)

# # Extract OOD features from each test loader
# test_loaders = {
#     "PGD": pgd_test_loader,
#     "FGSM": fgsm_test_loader,
#     "LSUN": lsun_test_loader,
#     "SVHN": DataLoader(svhn_test_dataset, batch_size=64, shuffle=False),
#     "iSUN": DataLoader(isun_test_dataset, batch_size=64, shuffle=False),
# }

# for name, loader in test_loaders.items():
#     ood_features, _ = extract_features(loader, model)
#     ood_features_dict[name] = ood_features

# # Dimensionality Reduction with PCA
# pca = PCA(n_components=2)
# pca_features_id = pca.fit_transform(features_id)
# pca_features_ood_dict = {name: pca.transform(ood_features) for name, ood_features in ood_features_dict.items()}

# # Dimensionality Reduction with t-SNE
# tsne = TSNE(n_components=2, perplexity=30, random_state=0)
# tsne_features_id = tsne.fit_transform(features_id)
# tsne_features_ood_dict = {name: tsne.fit_transform(ood_features) for name, ood_features in ood_features_dict.items()}

# # Plotting PCA and t-SNE visualizations
# unique_classes = sorted(set(labels_id))
# num_ood = len(pca_features_ood_dict)

# # Creating separate grids for PCA and t-SNE
# fig_pca, axes_pca = plt.subplots(len(unique_classes), num_ood, figsize=(15, 5 * len(unique_classes)))
# fig_tsne, axes_tsne = plt.subplots(len(unique_classes), num_ood, figsize=(15, 5 * len(unique_classes)))

# if len(unique_classes) == 1:
#     axes_pca = [axes_pca]  # Ensure axes is iterable for a single class
#     axes_tsne = [axes_tsne]

# for i, class_label in enumerate(unique_classes):
#     class_mask = labels_id == class_label
#     class_features_id_pca = pca_features_id[class_mask]
#     class_features_id_tsne = tsne_features_id[class_mask]

#     for j, (ood_name, pca_features_ood) in enumerate(pca_features_ood_dict.items()):
#         # PCA Plot
#         axes_pca[i][j].scatter(class_features_id_pca[:, 0], class_features_id_pca[:, 1], alpha=0.7, label=f"Class {class_label}")
#         axes_pca[i][j].scatter(pca_features_ood[:, 0], pca_features_ood[:, 1], alpha=0.7, label=f"{ood_name} (OOD)", c='red')
#         axes_pca[i][j].set_title(f"PCA: Class {class_label} vs {ood_name} (OOD)")
#         axes_pca[i][j].set_xlabel("PCA Dimension 1")
#         axes_pca[i][j].set_ylabel("PCA Dimension 2")
#         axes_pca[i][j].legend()

#         # t-SNE Plot
#         tsne_features_ood = tsne_features_ood_dict[ood_name]
#         axes_tsne[i][j].scatter(class_features_id_tsne[:, 0], class_features_id_tsne[:, 1], alpha=0.7, label=f"Class {class_label}")
#         axes_tsne[i][j].scatter(tsne_features_ood[:, 0], tsne_features_ood[:, 1], alpha=0.7, label=f"{ood_name} (OOD)", c='red')
#         axes_tsne[i][j].set_title(f"t-SNE: Class {class_label} vs {ood_name} (OOD)")
#         axes_tsne[i][j].set_xlabel("t-SNE Dimension 1")
#         axes_tsne[i][j].set_ylabel("t-SNE Dimension 2")
#         axes_tsne[i][j].legend()

# plt.tight_layout()
# plt.show()


In [ ]:
# import torch
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE

# # Assume `model` is your ResNet instance and `data_loader` is your DataLoader with 100 samples from 10 classes.

# # Initialize storage for features and labels
# features_4 = []
# labels = []
# layer_index = 4
# # Extract features from layer 1
# model.eval()
# with torch.no_grad():
#     for inputs, targets in train_loader:
#         inputs = inputs.cuda()
#         layer1_features = model.intermediate_forward(inputs, layer_index=layer_index)
#         # Flatten the features if they are multi-dimensional (e.g., [batch, channels, height, width])
#         layer1_features_flat = layer1_features.view(layer1_features.size(0), -1)
#         features_4.append(layer1_features_flat.cpu())
#         labels.append(targets.cpu())
#         if len(labels) >500:
#             break

# # Concatenate all features and labels
# features_4 = torch.cat(features_4, dim=0).numpy()
# labels = torch.cat(labels, dim=0).numpy()

# # Plotting
# fig, axes = plt.subplots(1, 2, figsize=(15, 7))
# # PCA
# pca = PCA(n_components=2)
# pca_features = pca.fit_transform(features_4)
# print("PCA Dope ")
# # PCA Plot
# axes[0].scatter(pca_features[:, 0], pca_features[:, 1], c=labels, cmap='tab10', alpha=0.7)
# axes[0].set_title("PCA Feature Visualization")
# axes[0].set_xlabel("Dimension 1")
# axes[0].set_ylabel("Dimension 2")

# # t-SNE
# tsne = TSNE(n_components=2, perplexity=30, random_state=0)
# tsne_features = tsne.fit_transform(features_4)




# # t-SNE Plot
# axes[1].scatter(tsne_features[:, 0], tsne_features[:, 1], c=labels, cmap='tab10', alpha=0.7)
# axes[1].set_title("t-SNE Feature Visualization")
# axes[1].set_xlabel("Dimension 1")
# axes[1].set_ylabel("Dimension 2")

# plt.colorbar(plt.cm.ScalarMappable(cmap='tab10'), ax=axes, location='right', label="Class")
# plt.tight_layout()

# plt.savefig("layer 4.png", dpi=300, bbox_inches='tight')

# plt.show()

In [ ]:
# import torch
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE

# # Assume `model` is your ResNet instance and `data_loader` is your DataLoader with 100 samples from 10 classes.

# # Initialize storage for features and labels
# features_3 = []
# labels = []
# layer_index = 3
# # Extract features from layer 1
# model.eval()
# with torch.no_grad():
#     for inputs, targets in train_loader:
#         inputs = inputs.cuda()
#         layer1_features = model.intermediate_forward(inputs, layer_index=layer_index)
#         # Flatten the features if they are multi-dimensional (e.g., [batch, channels, height, width])
#         layer1_features_flat = layer1_features.view(layer1_features.size(0), -1)
#         features_3.append(layer1_features_flat.cpu())
#         labels.append(targets.cpu())
#         if len(labels) >500:
#             break

# # Concatenate all features and labels
# features_3 = torch.cat(features_3, dim=0).numpy()
# labels = torch.cat(labels, dim=0).numpy()

# # Plotting
# fig, axes = plt.subplots(1, 2, figsize=(15, 7))
# # PCA
# pca = PCA(n_components=2)
# pca_features = pca.fit_transform(features_3)
# print("PCA Dope ")
# # PCA Plot
# axes[0].scatter(pca_features[:, 0], pca_features[:, 1], c=labels, cmap='tab10', alpha=0.7)
# axes[0].set_title("PCA Feature Visualization layer 3")
# axes[0].set_xlabel("Dimension 1")
# axes[0].set_ylabel("Dimension 2")

# # t-SNE
# tsne = TSNE(n_components=2, perplexity=30, random_state=0)
# tsne_features = tsne.fit_transform(features_3)




# # t-SNE Plot
# axes[1].scatter(tsne_features[:, 0], tsne_features[:, 1], c=labels, cmap='tab10', alpha=0.7)
# axes[1].set_title("t-SNE Feature Visualization layer 3")
# axes[1].set_xlabel("Dimension 1")
# axes[1].set_ylabel("Dimension 2")

# plt.colorbar(plt.cm.ScalarMappable(cmap='tab10'), ax=axes, location='right', label="Class")
# plt.tight_layout()

# plt.savefig("layer 3.png", dpi=300, bbox_inches='tight')

# plt.show()

In [ ]:
# import torch
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE

# # Assume `model` is your ResNet instance and `data_loader` is your DataLoader with 100 samples from 10 classes.

# # Initialize storage for features and labels
# features_2 = []
# labels = []
# layer_index = 2
# # Extract features from layer 1
# model.eval()
# with torch.no_grad():
#     for inputs, targets in train_loader:
#         inputs = inputs.cuda()
#         layer1_features = model.intermediate_forward(inputs, layer_index=layer_index)
#         # Flatten the features if they are multi-dimensional (e.g., [batch, channels, height, width])
#         layer1_features_flat = layer1_features.view(layer1_features.size(0), -1)
#         features_2.append(layer1_features_flat.cpu())
#         labels.append(targets.cpu())
#         if len(labels) >500:
#             break

# # Concatenate all features and labels
# features_2 = torch.cat(features_2, dim=0).numpy()
# labels = torch.cat(labels, dim=0).numpy()

# # Plotting
# fig, axes = plt.subplots(1, 2, figsize=(15, 7))
# # PCA
# pca = PCA(n_components=2)
# pca_features = pca.fit_transform(features_2)
# print("PCA Dope ")
# # PCA Plot
# axes[0].scatter(pca_features[:, 0], pca_features[:, 1], c=labels, cmap='tab10', alpha=0.7)
# axes[0].set_title("PCA Feature Visualization layer 2")
# axes[0].set_xlabel("Dimension 1")
# axes[0].set_ylabel("Dimension 2")

# # t-SNE
# tsne = TSNE(n_components=2, perplexity=30, random_state=0)
# tsne_features = tsne.fit_transform(features_2)




# # t-SNE Plot
# axes[1].scatter(tsne_features[:, 0], tsne_features[:, 1], c=labels, cmap='tab10', alpha=0.7)
# axes[1].set_title("t-SNE Feature Visualization layer 2")
# axes[1].set_xlabel("Dimension 1")
# axes[1].set_ylabel("Dimension 2")

# plt.colorbar(plt.cm.ScalarMappable(cmap='tab10'), ax=axes, location='right', label="Class")
# plt.tight_layout()

# plt.savefig("layer 2.png", dpi=300, bbox_inches='tight')

# plt.show()

In [ ]:
# import torch
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE

# # Assume `model` is your ResNet instance and `data_loader` is your DataLoader with 100 samples from 10 classes.

# # Initialize storage for features and labels
# features_1 = []
# labels = []
# layer_index = 1
# # Extract features from layer 1
# model.eval()
# with torch.no_grad():
#     for inputs, targets in train_loader:
#         inputs = inputs.cuda()
#         layer1_features = model.intermediate_forward(inputs, layer_index=layer_index)
#         # Flatten the features if they are multi-dimensional (e.g., [batch, channels, height, width])
#         layer1_features_flat = layer1_features.view(layer1_features.size(0), -1)
#         features_1.append(layer1_features_flat.cpu())
#         labels.append(targets.cpu())
#         if len(labels) >500:
#             break

# # Concatenate all features and labels
# features_1 = torch.cat(features_1, dim=0).numpy()
# labels = torch.cat(labels, dim=0).numpy()

# # Plotting
# fig, axes = plt.subplots(1, 2, figsize=(15, 7))
# # PCA
# pca = PCA(n_components=2)
# pca_features = pca.fit_transform(features_1)
# print("PCA Dope ")
# # PCA Plot
# axes[0].scatter(pca_features[:, 0], pca_features[:, 1], c=labels, cmap='tab10', alpha=0.7)
# axes[0].set_title("PCA Feature Visualization layer 1")
# axes[0].set_xlabel("Dimension 1")
# axes[0].set_ylabel("Dimension 2")

# # t-SNE
# tsne = TSNE(n_components=2, perplexity=30, random_state=0)
# tsne_features = tsne.fit_transform(features_1)




# # t-SNE Plot
# axes[1].scatter(tsne_features[:, 0], tsne_features[:, 1], c=labels, cmap='tab10', alpha=0.7)
# axes[1].set_title("t-SNE Feature Visualization layer 1")
# axes[1].set_xlabel("Dimension 1")
# axes[1].set_ylabel("Dimension 2")

# plt.colorbar(plt.cm.ScalarMappable(cmap='tab10'), ax=axes, location='right', label="Class")
# plt.tight_layout()

# plt.savefig("layer 1.png", dpi=300, bbox_inches='tight')

# plt.show()